In [159]:
!pip install reportlab
!pip install svglib

In [358]:
import pandas as pd

from reportlab.lib.pagesizes import LETTER
from reportlab.pdfgen import canvas
from reportlab.lib.colors import black, HexColor, white, grey
from reportlab.graphics import renderPDF

In [360]:
# goal: generate PDF files pre-filled with information. Write a script to do all of the conditional formatting for 
# each attendee's badge and render it into a PDF document. This would be things like filling in their name, 
# company, pronouns, photo release indication, etc... based on a CSV dump of the data from the ticketing provider

In [362]:
df = pd.read_csv("tito-export-format.csv")
df.head()

,Number,Ticket,Ticket Company Name,Ticket Job Title,Photo opt-out,Pronouns,What name would you like printed on your badge?,Would you like your pronouns printed on your badge?
0,116,Early Bird Corporate,The Aerospace Corporation,Project Leader,Opt-out,He / Him / His,Henry,NaN
1,117,Early Bird Individual,NaN,NaN,Opt-in,He/him,Albert Chae,NaN
2,118,Early Bird Individual,NaN,Sr. Software Engineer,Opt-in,He/Him,Austin Simmons,NaN
3,119,Early Bird Individual,Caltech,Senior Application Developer,Opt-in,She/Her,Cynthia Kiser,NaN
4,120,Donate to PyBeach,Caltech,Senior Application Developer,NaN,NaN,NaN,NaN


In [364]:
df.columns

Index(['Number', 'Ticket', 'Ticket Company Name', 'Ticket Job Title',
       'Photo opt-out', 'Pronouns',
       'What name would you like printed on your badge?',
       'Would you like your pronouns printed on your badge?'],
      dtype='object')

In [366]:
df['Ticket'].unique()

array(['Early Bird Corporate', 'Early Bird Individual',
       'Donate to PyBeach', 'Early Bird Student', 'Individual', 'Student',
       'Unemployed', 'Corporate'], dtype=object)

In [368]:
# ignore ticket type = Donate to PyBeach, which are donations
df_filtered = df[df['Ticket'] != 'Donate to PyBeach']

In [370]:
df_filtered['Photo opt-out'].unique()

array(['Opt-out', 'Opt-in', nan], dtype=object)

In [372]:
# ignore df rows where photo opt-out is null
df_filtered = df[df['Photo opt-out'].notna()]

In [374]:
# reset index
df_filtered = df_filtered.reset_index()

In [376]:
df_filtered['Number'].unique()

array([116, 117, 118, 119, 121, 122, 123, 125, 126, 127, 128, 129, 130,
        19, 135, 137, 139, 141,  28, 144, 146, 147, 149, 150, 151, 152,
       153, 155, 156, 157,  43, 161, 162, 163,  47,  48, 166, 167,  51,
        53,  54, 173])

In [378]:
df_filtered['Photo opt-out'].unique()

array(['Opt-out', 'Opt-in'], dtype=object)

In [423]:
df_filtered['Attendee'] = 'Attendee'
df_filtered

,index,Number,Ticket,Ticket Company Name,Ticket Job Title,Photo opt-out,Pronouns,What name would you like printed on your badge?,Would you like your pronouns printed on your badge?,Attendee
0,0,116,Early Bird Corporate,The Aerospace Corporation,Project Leader,Opt-out,He / Him / His,Henry,NaN,Attendee
1,1,117,Early Bird Individual,NaN,NaN,Opt-in,He/him,Albert Chae,NaN,Attendee
2,2,118,Early Bird Individual,NaN,Sr. Software Engineer,Opt-in,He/Him,Austin Simmons,NaN,Attendee
3,3,119,Early Bird Individual,Caltech,Senior Application Developer,Opt-in,She/Her,Cynthia Kiser,NaN,Attendee
4,5,121,Early Bird Individual,NaN,NaN,Opt-in,He / Him,Victor Hugo Solis,NaN,Attendee
5,6,122,Early Bird Student,NaN,NaN,Opt-in,He / Him,Bolun Thompson,NaN,Attendee
6,7,123,Early Bird Individual,Rouse Services,Director of Cloud Infrastructure,Opt-in,He / Him,Andrew Kurin,NaN,Attendee
7,9,125,Early Bird Individual,SoCal Python,Organizer,Opt-out,she/her,Esther,NaN,Attendee
8,10,126,Early Bird Individual,Flawless AI,Staff Platform Engineer,Opt-in,He / Him,Dorian,NaN,Attendee
9,11,127,Early Bird Individual,8th Light,Crafter,Opt-in,he / him,Ian G,NaN,Attendee


In [382]:

def return_fontsize_that_fits(badge_width, text, font_size):
    text_width = stringWidth(text, 'Helvetica', font_size)
    while text_width > badge_width:
        font_size -= 3
        text_width = stringWidth(text, 'Helvetica', font_size)
    return font_size

def standardize_pronouns(pronoun_str):
    if "/" in pronoun_str:
        pronouns = pronoun_str.split("/")
    else:
        pronouns = [pronoun_str]

    pronouns = [pro.lower().strip() for pro in pronouns]
    final_list = []
    
    if 'he' in pronouns or 'him' in pronouns or 'his' in pronouns:
        final_list.append('he')
    if 'she' in pronouns or 'her' in pronouns or 'hers' in pronouns:
        final_list.append('she')
    if 'they' in pronouns or 'them' in pronouns or 'theirs' in pronouns or 'their' in pronouns:
        final_list.append('they')

    # only she:
    if len(final_list) == 1:
        if final_list[0] == 'he':
            return 'He / Him'
        elif final_list[0] == 'she':
            return 'She / Hers'
        else:
            return 'They / Them'
    else:
        final = ""
        for item in final_list:
            final += item.capitalize() + " / "
        return final[:-2]
    
def format_pronouns(pronoun_str):
    cleaned = " ".join(pronoun_str.split())
    cleaned = re.sub(r"\s*/\s*", " / ", cleaned)
    formatted = " ".join(word.capitalize() for word in cleaned.split())
    return formatted

In [384]:
standardize_pronouns('He / him')

'He / Him'

In [429]:
from svglib.svglib import svg2rlg

badge_count = 0
badge_width = 4 * 72    # 2 inches
badge_height = 2.875 * 72   # 3 inches
width, height = LETTER
# 10 point margin on sides, 20 point on top bottom
badge_xys = [
    (10, 565),   # top left
    (316, 565), # top right
    (10, 338),   # middle left
    (316, 338), # middle right
    (10, 111),   # bottom left
    (316, 111)  # bottom right
]

# starting canvas
output_path = "badges_0.pdf"
c = canvas.Canvas(output_path, pagesize=LETTER)

for index, row in df_filtered.iterrows():
    if badge_count == 6:
        c.save()
        print(f"PDF saved to {output_path}")
        output_path = "badges_" + str(index) + ".pdf"
        # start a new sheet of badges
        c = canvas.Canvas(output_path, pagesize=LETTER)
        c.setFillColor(black)
        badge_count = 0
    name = row["What name would you like printed on your badge?"]
    if pd.isna(name):
        print(f'Error with row {index}: name is empty')
        continue
    if pd.isna(row['Number']):
        print(f'Error with row {index}: number is empty')
        continue
    
    # top left for the badge
    # 20 point margins
    x = badge_xys[badge_count][0]
    y = badge_xys[badge_count][1]
    
    # photo opt-out icon: top left
    if row['Photo opt-out'] == 'Opt-out':
        # c.drawImage('camera.png', x + 4, y + badge_height - 50, width=50, height=50, mask='auto')
        drawing = svg2rlg('no-photos.svg')
        logo_size = 50
        scale = min(logo_size / drawing.width, logo_size / drawing.height)
        drawing.width *= scale
        drawing.height *= scale
        drawing.scale(scale, scale)
        renderPDF.draw(drawing, c, x + 4, y + badge_height - 50)
    
    # pronouns: top right
    pronouns_option = row['Would you like your pronouns printed on your badge?']
    if pd.isna(pronouns_option) or pronouns_option == 'Yes':
        pronouns = row['Pronouns']
        # could be None
        if not pd.isna(pronouns) and pronouns != "-":
            c.setFont("Helvetica", 13)
            c.setFillColor(black)
            c.drawRightString(x + badge_width - 4, y + badge_height - 8, pronouns)

    drawing = svg2rlg('pybeach2025-fullcolor.svg')
    logo_size = 72
    scale = min(logo_size / drawing.width, logo_size / drawing.height)
    drawing.width *= scale
    drawing.height *= scale
    drawing.scale(scale, scale)

    renderPDF.draw(drawing, c, x + (badge_width - logo_size) / 2, y + badge_height - logo_size - 10)
    # name: center
    # clean up spacing in name
    cleaned_name = re.sub(r"\s+", " ", name).strip()
    font_size = return_fontsize_that_fits(badge_width, cleaned_name, 20)
    c.setFont("Helvetica-Bold", font_size)
    c.setFillColor(black)
    # c.drawCentredString(x + badge_width / 2, ((y + badge_height - logo_size - 10) + 130) / 2, cleaned_name)

    # Company name should only be printed on Corporate and Early Bird Corporate badges
    # options: nothing, just company, just title, or both
    typ = row["Ticket"]
    company = ""
    if typ == 'Early Bird Corporate' or typ == 'Corporate':
        company_name = row["Ticket Company Name"]
        company = company_name
        if not pd.isna(row["Ticket Job Title"]):
            title = row["Ticket Job Title"]
            company = f"{title}, " + company
    else:
        if not pd.isna(row["Ticket Job Title"]):
            title = row["Ticket Job Title"]
            company = f"{title} " + company

    if company == "":
        name_y = y + badge_height - 125
    else:
        name_y = y + badge_height - 110
        # else: empty line
    
    cleaned_name = re.sub(r"\s+", " ", name).strip()
    font_size = return_fontsize_that_fits(badge_width, cleaned_name, 20)
    c.setFont("Helvetica-Bold", font_size)
    c.setFillColor(black)
    c.drawCentredString(x + badge_width / 2, name_y, cleaned_name)

    if company != "":
        job_font_size = return_fontsize_that_fits(badge_width, company, 14)
        c.setFont("Helvetica", job_font_size)
        c.setFillColor(black)
        c.drawCentredString(x + badge_width / 2, y + badge_height - 130, company)

    
    

    #     job_title = row["Ticket Job Title"]
    # if not pd.isna(job_title):
    #     line = line - 20
    #     c.drawString(100, line, f"Job Title: {job_title}")
    event = 'PyBeach | September 27, 2025' 
    c.setFillColor(grey)
    c.setFont("Helvetica", 13)
    c.drawCentredString(x + badge_width / 2, y + badge_height - 165, event)

    if not pd.isna(row["Attendee"]):
        ribbon_height = 25
        c.setFillColor(HexColor("#337ab7"))
        c.rect(x, y + 5, badge_width, ribbon_height, stroke=0, fill=1)
        c.setFillColor(white)
        c.setFont("Helvetica-Bold", 14)
        c.drawCentredString(x + badge_width / 2, y + 12, row["Attendee"])
    else:
        print(f'Warning: {name} does not have a value for Attendee column.')
    
    badge_count += 1
# save any remaining canvas
c.save()
print(f"PDF saved to {output_path}")

PDF saved to badges_0.pdf
PDF saved to badges_6.pdf
PDF saved to badges_12.pdf
PDF saved to badges_18.pdf
PDF saved to badges_24.pdf
PDF saved to badges_30.pdf
PDF saved to badges_36.pdf


In [340]:
df_filtered[df_filtered['Photo opt-out'] == 'Opt-out']

,index,Number,Ticket,Ticket Company Name,Ticket Job Title,Photo opt-out,Pronouns,What name would you like printed on your badge?,Would you like your pronouns printed on your badge?
0,0,116,Early Bird Corporate,The Aerospace Corporation,Project Leader,Opt-out,He / Him / His,Henry,NaN
7,9,125,Early Bird Individual,SoCal Python,Organizer,Opt-out,she/her,Esther,NaN
10,12,128,Early Bird Student,Caltech,Research,Opt-out,He / Him,Michael Langmayr,NaN
11,13,129,Early Bird Student,Caltech,NaN,Opt-out,She / Her,Prakriti,NaN
27,39,155,Early Bird Student,NaN,NaN,Opt-out,He/him,Tony,NaN
38,50,51,Individual,SETI Institute/NASA Ames,Senior Science Systems Engineer,Opt-out,He/Him,"Andrew Annex, PhD.",NaN
